<a href="https://colab.research.google.com/github/vishalkumar014/task-board/blob/main/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting started with Mistral AI API

In [52]:
! pip install -q mistralai

In [53]:
import os
from mistralai import Mistral
api_key = 'ZMNIE55XjboSZRUzEfKfgr2YMjRWf7nX'

###Code with file upload


In [120]:
image_path = "/content/Resume.pdf"
client = Mistral(api_key=api_key)
uploaded_pdf = client.files.upload(
    file={
        "file_name": "uploaded_file.pdf",
        "content": open(image_path, "rb"),
    },
    purpose="ocr"
)

In [121]:
client.files.retrieve(file_id=uploaded_pdf.id)
signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)

In [122]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "what is the name in the document Return the name in short JSON object."
            },
            {
                "type": "document_url",
                "document_url":  signed_url.url
            }
        ]
    }
]

In [123]:
chat_response = client.chat.complete(
    model=model,
    messages=messages,
    response_format = {
          "type": "json_object",
      }
)
print(chat_response.choices[0].message.content)

{
  "name": "Vishal Kumar"
}


###Without file upload

In [112]:
import base64
import requests

def encode_image(image_path):
    """Encode the image to base64."""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {image_path} was not found.")
        return None
    except Exception as e:  # Added general exception handling
        print(f"Error: {e}")
        return None

image_path = "/content/Resume.pdf"
base64_image = encode_image(image_path)

In [113]:
ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "document_url",
        "document_url": f"data:application/pdf;base64,{base64_image}"
    }
)


In [117]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """ what is the name in the document Return the name in short JSON object.
                  {ocr_response}
                """.format(ocr_response = ocr_response)
            },
        ]
    }
]

In [118]:
chat_response = client.chat.complete(
    model=model,
    messages=messages,
    response_format = {
          "type": "json_object",
      }
)

In [119]:
print(chat_response.choices[0].message.content)

{"name": "Vishal kumar"}
